1종 오류와 2종 오류 처리
- 어떤 유형의 오류가 다른 유형의 오류보다 중요할 수 있음
- PDF 파일과 같이 파일이 매우 일반적이지만 거의 드문 악성코드인 경우 원하는 오탐율은 0.01%와 같이 매우 낮음
- 한계점(threshold)을 사용하여 원하는 오탐율의 한계를 넘지 않도록 하는 방법 사용
- 데이터셋을 가져와 분류기 훈련하고 나서 오탐율 제약 조건 충족시키고자 한계점 조정

In [1]:
# 데이터셋을 가져오고 원하는 오탐율이 1% 이하라는 것 지정
import numpy as np
from scipy import sparse
import scipy

X_train = scipy.sparse.load_npz("../Resources/training_data.npz")
y_train = np.load("../Resources/training_labels.npy")
X_test = scipy.sparse.load_npz("../Resources/test_data.npz")
y_test = np.load("../Resources/test_labels.npy")
desired_FPR = 0.01

In [2]:
# 오탐율(False Positive Rate)과 정탐율(True Positive Rate) 계산하는 함수 만듦
from sklearn.metrics import confusion_matrix


def FPR(y_true, y_pred):
    """Calculate the False Positive Rate."""
    CM = confusion_matrix(y_true, y_pred)
    TN = CM[0][0]
    FP = CM[0][1]
    return FP / (FP + TN)


def TPR(y_true, y_pred):
    """Calculate the True Positive Rate."""
    CM = confusion_matrix(y_true, y_pred)
    TP = CM[1][1]
    FN = CM[1][0]
    return TP / (TP + FN)

In [3]:
# 한계점을 사용하여 확률 벡터를 bool 벡터로 변환하는 함수 만듦
def perform_thresholding(vector, threshold):
    """Threshold a vector."""
    return [0 if x >= threshold else 1 for x in vector]

In [4]:
# XGBoost 모델을 훈련하고 훈련 데이터에 대한 예측 확률 계산
from xgboost import XGBClassifier

clf = XGBClassifier()
clf.fit(X_train, y_train)
clf_pred_prob = clf.predict_proba(X_train)

In [5]:
# 예측 확률 벡터를 확인
print("Probabilities look like so:")
print(clf_pred_prob[0:5])
print()

Probabilities look like so:
[[9.9711573e-01 2.8842732e-03]
 [9.9930882e-01 6.9116493e-04]
 [9.9943042e-01 5.6955573e-04]
 [9.8729378e-01 1.2706242e-02]
 [9.0440232e-01 9.5597707e-02]]



In [8]:
# 1000개의 다른 한계점 값에 대해 반복하면서 각 한계점에 대한 오탐율을 계산하고, FPR<=DesiredFDP을 만족할 때 한계점 값을 계산
M = 1000
print("Fitting threshold:")
for t in reversed(range(M)):
    scaled_threshold = float(t) / M
    thresholded_prediction = perform_thresholding(clf_pred_prob[:, 0], scaled_threshold)
    FPR_=FPR(y_train, thresholded_prediction)
    TPR_=TPR(y_train, thresholded_prediction)
    print(f"{t:03}번째 시도 한계점: {scaled_threshold:.3f}, 오탐율:{FPR_:.4f}, 정탐율: {TPR_:.4f}")
    if FPR_ <= desired_FPR:
        print()
        print("Selected threshold: ")
        print(scaled_threshold)
        break

Fitting threshold:
999번째 시도 한계점: 0.999, 오탐율:0.4682, 정탐율: 1.0000
998번째 시도 한계점: 0.998, 오탐율:0.3364, 정탐율: 1.0000
997번째 시도 한계점: 0.997, 오탐율:0.2182, 정탐율: 1.0000
996번째 시도 한계점: 0.996, 오탐율:0.1864, 정탐율: 1.0000
995번째 시도 한계점: 0.995, 오탐율:0.1682, 정탐율: 1.0000
994번째 시도 한계점: 0.994, 오탐율:0.1500, 정탐율: 1.0000
993번째 시도 한계점: 0.993, 오탐율:0.1409, 정탐율: 1.0000
992번째 시도 한계점: 0.992, 오탐율:0.1273, 정탐율: 1.0000
991번째 시도 한계점: 0.991, 오탐율:0.1182, 정탐율: 1.0000
990번째 시도 한계점: 0.990, 오탐율:0.1136, 정탐율: 1.0000
989번째 시도 한계점: 0.989, 오탐율:0.1136, 정탐율: 1.0000
988번째 시도 한계점: 0.988, 오탐율:0.1136, 정탐율: 1.0000
987번째 시도 한계점: 0.987, 오탐율:0.1091, 정탐율: 1.0000
986번째 시도 한계점: 0.986, 오탐율:0.1091, 정탐율: 1.0000
985번째 시도 한계점: 0.985, 오탐율:0.1091, 정탐율: 1.0000
984번째 시도 한계점: 0.984, 오탐율:0.1000, 정탐율: 1.0000
983번째 시도 한계점: 0.983, 오탐율:0.1000, 정탐율: 1.0000
982번째 시도 한계점: 0.982, 오탐율:0.1000, 정탐율: 1.0000
981번째 시도 한계점: 0.981, 오탐율:0.1000, 정탐율: 1.0000
980번째 시도 한계점: 0.980, 오탐율:0.1000, 정탐율: 1.0000
979번째 시도 한계점: 0.979, 오탐율:0.1000, 정탐율: 1.0000
978번째 시도 한계점: 0.978, 오탐율:0.1000, 정탐율